In [1]:
import preprocessing
import pandas as pd
from data_reading.phyphox import get_experiments, read_experiment
listening_rate = 20

experiments = get_experiments()
data_frame = read_experiment(experiments[0])
data_frame = preprocessing.align_data(data_frame, listening_rate=listening_rate, reference_sensor=None)

video_length = 340
sample_rate = len(data_frame) / video_length
# TODO: load annotations
y = pd.read_csv("../../data/phyphox/long recording/annotations.tsv", delimiter="\t")
y = y.iloc[:, [3,5,8]]
y.columns = ["start", "end", "label"]
# iterate over the annotations and split the timeseries in chunks
chunks = [data_frame.iloc[int(annotation["start"]*sample_rate):int(annotation["end"]*sample_rate)] for i, annotation in y.iterrows()]

/Users/julio/Documents/Uni/5_Master/Master Project/mp-mental-health-2020/src/preprocessing/_data_frame.py:31: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  time_delta_index = pd.TimedeltaIndex(timestamp_to_date, unit=output_timestamp_unit)


In [12]:
# TODO: replace label string values with indices
labels = y.loc[:, "label"].unique()
label_ids = { l: i for l,i in zip(labels, range(0,len(labels))) }

y = y.replace(label_ids)
for c,cl in zip(chunks, y.iterrows()):
    c["activity"] = cl[1]["label"]
    c["action_id"] = cl[0]

     start      end  label
0   19.489   20.754      0
1   23.168   24.149      0
2   39.866   40.243      1
3  303.077  304.934      2
4  312.913  314.971      2


/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [17]:
pd.concat(chunks)

,acceleration_x,acceleration_y,acceleration_z,gyroscope_x,gyroscope_y,gyroscope_z,activity,action_id
00:00:18.679897,0.035320,-0.013337,0.026752,-0.023912,-0.008865,0.006441,0.0,0
00:00:18.699897,0.059538,0.018115,-0.006217,-0.016727,-0.003208,0.015903,0.0,0
00:00:18.719897,-0.024354,0.020793,0.060347,-0.008347,-0.001332,0.018921,0.0,0
00:00:18.739897,0.013580,0.013835,0.009312,-0.008660,0.001037,0.014761,0.0,0
00:00:18.759897,0.084536,0.007026,0.070932,-0.009054,-0.004316,0.018061,0.0,0
...,...,...,...,...,...,...,...,...
00:05:01.799897,0.973294,0.599652,-0.882202,0.129783,-0.905352,-0.760374,2.0,4
00:05:01.819897,1.009741,0.447766,-0.913241,0.179277,-0.826804,-0.738206,2.0,4
00:05:01.839897,0.973053,0.304662,-1.037655,0.234332,-0.764604,-0.700717,2.0,4
00:05:01.859897,0.862463,0.108927,-1.131094,0.286549,-0.707382,-0.646120,2.0,4


In [16]:
%load_ext autoreload
%autoreload 2

labels = y.loc[:, "label"].squeeze()
print(labels)
from features._timeseries_feature_extraction import extract_timeseries_features
features = extract_timeseries_features(pd.concat(chunks), labels)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0    0
1    0
2    1
3    2
4    2
Name: label, dtype: int64


Feature Extraction: 100%|██████████| 18/18 [00:01<00:00, 17.09it/s]


In [14]:
features

variable
id
0
1
2
3
4
